# Water Quality Parameters

This notebook aims at computing the descriptive statistics for the WQP maps, or any raster, by defining specific features from which these statistics are requested (located within the [in](./in) folder). In addition, the analysis of the WQP maps will consider de extraction of sampling data inside the pixels to review the estimates.

In [ ]:
# Styling notebook

# System
import os
import sys

# Import scripts libraries for the project
sys.path.append('./src/python')

# Import the function to update the notebook style
from nbConfig import (css_styling)


css_styling()

In [ ]:
from matplotlib import rcParams
font = {'family' : 'Georgia',
#         'weight' : 'bold',
        'size'   : 12}

## 0. Import Libraries

In [ ]:
# Data
import numpy as np
import pandas as pd

# Spatial Data
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

# Data visualization
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn as sns
from rasterio.plot import show_hist
import ipywidgets as wg

# Import custom libraries
import wqpFunctions as wqp

Define the working directory for the WQP processing

In [ ]:
# Current Working Directories
cwd = {
    'local': '.',
    'in': './in/wqp/',
    'out': './out/outlier_rejection',
    'vector': './vector',
}

Import the vector files for data extraction of the WQP maps

In [ ]:
# Layer dataset
gdf_lakes = gpd.read_file(os.path.join(cwd['vector'],'simile_laghi','simile_laghi.shp'))
# Buoy position in the lake
gdf_buoy = gpd.read_file(os.path.join(cwd['vector'],'boa_sample_points','boa_sample_points.shp'))
# Random sampling points
gdf_sample = gpd.read_file(os.path.join(cwd['vector'],'random_points','random_points.shp'))
# Buffer of the extent of the lakes
gdf_buffer = gpd.read_file(os.path.join(cwd['vector'],'simile_laghi_bboxes','simile_laghi_bboxes.shp'))

Here, different vector layers are needed to carry out the following statistical analysis of the results form the WQP production. The next statement presents the lake's delineation along with the sampling points. The lake's delineation, extracted from the [Carta Tecnica Regionale 1:10.000 - CT10](https://www.geoportale.regione.lombardia.it/metadati?p_p_id=detailSheetMetadata_WAR_gptmetadataportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_detailSheetMetadata_WAR_gptmetadataportlet_uuid=%7BD42AE439-06FE-4E4E-93CF-ECDC9B564118%7D) available at the [Lombardy Region GeoPortal](https://www.geoportale.regione.lombardia.it/), is used for the computation of the descriptive statistics of the lakes WQP parameters per lake. While the second vector file are ranodmly generated random point for the statistical analysis of the WQP maps.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.rc('font', **font)
gdf_lakes.plot(ax = ax, color= 'blue', edgecolor='k', alpha = 0.3)
gdf_sample.plot(ax = ax, color='red')
gdf_buoy.plot(ax = ax, color='green')

scale = ScaleBar(
    dx=1,
    location='lower right',  # in relation to the whole plot
    label_loc='left', scale_loc='bottom',  # in relation to the line
)

ax.set_title('Sampling Points')
ax.tick_params(labelsize=12)
ax.set_xlabel('[m]')
ax.set_ylabel('[m]')
ax.add_artist(scale)
ax.legend(['Sampling Points','Buoys Location'])
ax.ticklabel_format(axis='both', style='sci', scilimits=(5,4))

ax.arrow(4.6e5,5.11e6,0,1000)
ax.grid()

plt.savefig('./figs/sampling_points.png',dpi=300,Transparent=True)

Bounding boxes for the statistical analysis for each lake

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.rc('font', **font)
gdf_buffer.plot(ax = ax, edgecolor='k', alpha = 0.5)
gdf_lakes.plot(ax = ax, color= 'blue', edgecolor='k', alpha = 0.3)
# gdf_sample.plot(ax = ax, color='red')
# gdf_buoy.plot(ax = ax, color='green')

scale = ScaleBar(
    dx=1,
    location='lower right',  # in relation to the whole plot
    label_loc='left', scale_loc='bottom',  # in relation to the line
)

ax.set_title('Lakes bounding boxes extend - buffer 600m')
ax.tick_params(labelsize=12)
ax.set_xlabel('[m]')
ax.set_ylabel('[m]')
ax.add_artist(scale)
ax.legend(['Buffer'])
ax.ticklabel_format(axis='both', style='sci', scilimits=(5,4))

ax.arrow(4.6e5,5.11e6,0,1000)
ax.grid()

plt.savefig('./figs/bounding_boxes.png',dpi=300,Transparent=True)

Review the contents of the folder

### Select WQP and Outlier Rejection Method

The outlier rejection methods implemented within this work consider different filters. For the production of the WQP maps, the 3$\sigma$ has been selected as the default filter for the identification and extraction of the outliers from the WQP outputs.

In [ ]:
SENSOR_LIST = ['S3','L8','EUMETSAT']
sensor = wg.Select(
    options = SENSOR_LIST,
    value = SENSOR_LIST[1],
    description = 'Select the sensor:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='20%', height='100px'),
)

In [ ]:
WQP_LIST = ['chl','tsm','lswt']
wqpMaps = wg.Select(
    options = WQP_LIST,
    value = WQP_LIST[2],
    description = 'Select the wqp:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='15%', height='100px'),
)

In [ ]:
OUTLIERS_METHOD_LIST = ['IQR','2Sigma','3Sigma']
outlierMethods = wg.Select(
    options = OUTLIERS_METHOD_LIST,
    value = OUTLIERS_METHOD_LIST[0],
    description = 'Select the outlier method:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='30%', height='100px'),
)

In [ ]:
REF_PROD_LIST = ['wqp','wqp_cloud_mask','wqp_no_clip','wqp_no_mask','wqp_high_clouds','wqp_mid_high_clouds', 'full_outlierRejection']#last 2 L8
refProducts = wg.Select(
    options = REF_PROD_LIST,
    value = REF_PROD_LIST[4],
    description = 'Select ref product:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='35%', height='100px'),
)

In [ ]:
wg.HBox([sensor,wqpMaps, outlierMethods,refProducts])

## 1. WQP Descriptive Statistics and Sampling Points

In [ ]:
wqp_parameter = 'chl'
# in_path = f'./temp/wqp_maps/original/{wqp_parameter}'
# out_path = f'./temp/wqp_maps/original/{wqp_parameter}'
# in_path = f'./out/outlier_rejection/wqp/3Sigma/lswt'
# out_path = f'./out/outlier_rejection/wqp/3Sigma/lswt'
# in_path = './in/wqp/L8/wqp_high_clouds/lswt'
# out_path = './in/wqp/L8/wqp_high_clouds/lswt'
# in_path = './temp/lswt'
# out_path = './temp/lswt'


In [ ]:
# rename_files = os.listdir(out_path)
# rename_files
# for f in rename_files:
#     if f.endswith('tif'):
#         name = '_'.join(f.split('_')[:-1])+'.tif'
#         os.rename(f'{out_path}/{f}', f'{out_path}/{name}')

In [ ]:
# in_path = os.path.join(f'./out/image_coregistration/wqp/full_coregistered_or/3Sigma/{wqp_parameter}')
# out_path = os.path.join(f'./out/image_coregistration/wqp/full_coregistered_or/3Sigma/{wqp_parameter}/outliers')

In [ ]:
# in_path = os.path.join(f'./in/wqp/S3/wqp/{wqp_parameter}')
# in_path = os.path.join(f'./in/wqp/EUMETSAT/{wqp_parameter}')
# in_path = os.path.join('./extract_lswt')
# in_path = os.path.join('./in/data/wqp/L8/wqp_high_clouds/lswt/cropped')
# in_path = os.path.join('./out/OutlierRejection/wqp/3Sigma/lswt')
# in_path = os.path.join(cwd['out'],'wqp',outlierMethods.value, wqpMaps.value)
# in_path = os.path.join('./out/Coregistration/wqp/S3/full/chl')
in_path = os.path.join(f'./out/image_coregistration/wqp/full_coregistered/{wqp_parameter}')
# in_path = os.path.join(f'./extra/diff_maps/{wqp_parameter}')
# in_path = os.path.join('./out/Coregistration/wqp/S3/full_outlierRejection/3Sigma/tsm')
# out_path = os.path.join(f'./in/wqp/S3/wqp/{wqp_parameter}/cropped')
# out_path = os.path.join(f'./in/wqp/EUMETSAT/{wqp_parameter}/cropped')
# out_path = os.path.join(cwd['out'],'wqp',outlierMethods.value, wqpMaps.value)
# out_path = os.path.join('./out/OutlierRejection/wqp/3Sigma/lswt')
# out_path = os.path.join('./out/Coregistration/wqp/S3/full_outlierRejection/3Sigma/tsm')
out_path = os.path.join(f'./out/image_coregistration/wqp/full_coregistered/{wqp_parameter}/cropped')
# out_path = os.path.join(f'./extra/diff_maps/{wqp_parameter}/cropped')
# out_path = os.path.join('./extract_lswt')


issue_files = []
arr = []
for root, dirs, files in os.walk(out_path):
    for file in files:
        try:
            print(file)
            # Verify check if the stats and products have been computed
            if ((file.endswith('_L1.tif')) ):
                # if (file.split('.') not in os.listdir(in_path)):
                    f = os.path.join(out_path,file)
                    # Read file
                    src = wqp.wqp(f)
                    src.readWQP()
                    # print(src.name)
#                     src.name
                    # Export dataset (check no data values)
                    # t = src.image.read(1)
                    # t = np.nan_to_num(t)
                    # t[t<=0] = np.nan
                    # src.writeWQP(os.path.join(out_path,src.name+'.tif'),t)

                    # Extract information from sampling points
                    # src.extractSamplePoints(gdf_sample)
                    # src.extractSamplePoints(gdf_buoy)
                    # src.samplePoint.to_csv(os.path.join('./out','sample_points',src.name+'.csv'))

                    # Compute statistics for the lakes polygons
                    src.computeStatistics(gdf_buffer, 'Nome',"count min mean max median std  percentile_25 percentile_50 percentile_75",-9999)

                    # Format output
                    df = wqp.wqp.exportWQPFormatStats(src)
                    arr.append(df)
#                     # Export the statistics result to a file (append data if existing)
#                     out_file = os.path.join(out_path,f'lakesStats_{wqp_parameter}.csv')
# #                     print(out_path,f'lakesStats_{wqpMaps.value}.csv')
#                     if os.path.exists(out_file):
#                         df.to_csv(os.path.join(out_path,f'lakesStats_{wqp_parameter}.csv'),mode='a', header=False)
#                     else:
#                         df.to_csv(os.path.join(out_path,f'lakesStats_{wqp_parameter}.csv')) 
        except:
            issue_files.append(file)
with open(os.path.join(out_path,'issue_maps.txt'), 'w') as f:
    f.write('\n'.join(issue_files))

In [ ]:
df = pd.concat(arr)
df.head()

In [ ]:
# df = pd.concat(arr)
# df.to_csv(os.path.join(out_path,f'lakesStats_{wqp_parameter}.csv')) 
df.to_csv(os.path.join(out_path,f'lakesStats_{wqp_parameter}.csv'), encoding='utf-8',decimal='.') 

In [ ]:
os.path.join(out_path,f'lakesStats_{wqp_parameter}.csv')

## 2. Outlier Rejection Methods

Create the outliers "free" maps based on the selected wqpMaps and outlierMethods selected above

In [ ]:
wqp_parameter = 'chl'
outlier_method = 'IQR'
# in_path = os.path.join(cwd['in'],sensor.value,refProducts.value,wqpMaps.value)
# in_path = './temp/wqp_maps/lswt'
# out_path = './temp/wqp_maps/lswt_OR'
in_path = os.path.join(f'./out/image_coregistration/wqp/full_coregistered_or/{outlier_method}/{wqp_parameter}')
out_path = os.path.join(f'./out/image_coregistration/wqp/full_coregistered_or/{outlier_method}/{wqp_parameter}/outliers')
# in_path = os.path.join(f'./out/outlier_rejection/wqp/{outlier_method}/lswt')
# out_path = os.path.join(f'./out/outlier_rejection/wqp/{outlier_method}/lswt/outliers')

In [ ]:
in_path

In [ ]:
out_path

In [ ]:
# https://stackoverflow.com/questions/229186/os-walk-without-digging-into-directories-below
def walklevel(some_dir, level=1):
    some_dir = some_dir.rstrip(os.path.sep)
    assert os.path.isdir(some_dir)
    num_sep = some_dir.count(os.path.sep)
    for root, dirs, files in os.walk(some_dir):
        yield root, dirs, files
        num_sep_this = root.count(os.path.sep)
        if num_sep + level <= num_sep_this:
            del dirs[:]

In [ ]:
a = walklevel(in_path, level=1)

In [ ]:
# in_path = os.path.join(cwd['in'],sensor.value,refProducts.value,wqpMaps.value)
# out_path = os.path.join(cwd['out'],'wqp',outlierMethods.value, wqpMaps.value)
shared_maps = list(set(os.listdir(in_path)).intersection(os.listdir(out_path)))
missing_maps = list(set(os.listdir(in_path)) ^ set(os.listdir(out_path)))

d = []
for root, dirs, files in os.walk(in_path):
    l = os.listdir(os.path.join(root))
    if root.split('/')[-1]!='outliers':
        for file in l:
        # for file in missing_maps:
            if ((file.endswith('.tif'))):
                f = os.path.join(in_path,file)
                try:
                    # Read file
                    print(f)
                    src = wqp.wqp(f)
                    src.readWQP()
                    # Crop the lakes by the lakes 
                    src.cropRasterByFeatures(os.path.join(cwd['vector'],'simile_laghi','simile_laghi.shp'),'Nome')

                    # Compute statistics for the lakes polygons
                    src.computeStatistics(gdf_lakes, 'Nome',"count min mean max median std percentile_25 percentile_50 percentile_75",0)

                    # Apply the outlier rejection method
                    src.outlierRejection(method=outlierMethods.value, minLower=0, maxUpper = 100)

                    # # Export the outlier rejection products
                    # src.mergeRasterCollectionsExport(src.raster_collection, os.path.join(out_path, src.name + '.tif'))
                    # src.mergeRasterCollectionsExport(src.outliers_collection, os.path.join(out_path,'outliers', src.name + '.tif'))
                    src.mergeRasterCollectionsExport(src.outliers_collection, os.path.join(out_path, src.name + '.tif'))
                    print(src.outliers_collection, os.path.join(out_path,'outliers', src.name + '.tif'))

                    # Export outliers data
                    df = wqp.wqp.exportWQPFormatStatsOutliers(src)
                    d.append(df)

                    # Export the statistics result to a file (append data if existing)
                    out_file = os.path.join(out_path,'outliers',f'lakesStats_{outlierMethods.value}_{wqpMaps.value}.csv')

                    # if os.path.exists(out_file):
                    #     df.to_csv(os.path.join(out_path,'outliers',f'lakesStats_{outlierMethods.value}_{wqpMaps.value}.csv'),mode='a', header=False)
                    # else:
                    #     df.to_csv(os.path.join(out_path,'outliers',f'lakesStats_{outlierMethods.value}_{wqpMaps.value}.csv')) 
                except:
                    # Open a file with access mode 'a'
                    file_object = open(os.path.join(out_path,f'error_images_{sensor.value}.txt'), 'a')
                    # Append 'hello' at the end of file
                    file_object.write(src.name)
                    file_object.write("\n")
                    # Close the file
                    file_object.close()

In [ ]:
result_df = pd.concat(d)
result_df = result_df.replace(-9999, np.nan)
result_df.loc[result_df['upperBound_Como'].isna(),['lowerBound_Como','countLower_Como','countUpper_Como','countTotal_Como','percValid_Como','percOutliers_Como']]=np.nan
result_df.loc[result_df['upperBound_Maggiore'].isna(),['lowerBound_Maggiore','countLower_Maggiore','countUpper_Maggiore','countTotal_Maggiore','percValid_Maggiore','percOutliers_Maggiore']]=np.nan
result_df.loc[result_df['upperBound_Lugano'].isna(),['lowerBound_Lugano','countLower_Lugano','countUpper_Lugano','countTotal_Lugano','percValid_Lugano','percOutliers_Lugano']]=np.nan

In [ ]:
result_df.head()

In [ ]:
result_df.to_csv(os.path.join(out_path,f'{wqp_parameter}_{outlier_method}_outliers.csv'), encoding='utf-8', decimal='.')

## Run the following procedure to perform the outlier rejection for the coregistered images

In [ ]:
in_path = os.path.join('./out/images_coregistration/wqp','full_coregistered',wqpMaps.value)
out_path = os.path.join('./out/images_coregistration/wqp','full_coregistered_or',outlierMethods.value,wqpMaps.value)
print(in_path)
print(out_path)

In [ ]:
# TODO: apply the outlier rejection method to the missing datasets
cor_path = './out/image_coregistration/wqp'
# OUTLIER REJECTION FOR THE WQP COREGISTRERED MAPS

in_path = os.path.join(cor_path,'full_coregistered',wqpMaps.value)
out_path = os.path.join(cor_path,'full_coregistered_or',outlierMethods.value,wqpMaps.value)
print(in_path)
print(out_path)
for root, dirs, files in os.walk(in_path):
    for file in files:
        if ((file.endswith('.tif')) & ([]==dirs)):
            print(in_path,file)
            f = os.path.join(in_path,file)
            try:
                # Read file
                src = wqp.wqp(f)
                src.readWQP()
                # Crop the lakes by the lakes 
                src.cropRasterByFeatures(os.path.join(cwd['vector'],'simile_laghi','simile_laghi.shp'),'Nome')

                # Compute statistics for the lakes polygons
                src.computeStatistics(gdf_lakes, 'Nome',"count min mean max median std percentile_25 percentile_50 percentile_75",0)
                print(src.name)
                # Apply the outlier rejection method
                src.outlierRejection(method=outlierMethods.value, minLower=0, maxUpper = 30)    

                # Export the outlier rejection products
                src.mergeRasterCollectionsExport(src.raster_collection, os.path.join(out_path, src.name + '.tif'))
                src.mergeRasterCollectionsExport(src.outliers_collection, os.path.join(out_path,'outliers', src.name + '.tif'))

                # Export outliers data
                df = wqp.wqp.exportWQPFormatStatsOutliers(src)

                # Export the statistics result to a file (append data if existing)
                out_file = os.path.join(out_path,'outliers',f'lakesStats_{outlierMethods.value}_{wqpMaps.value}.csv')
                if os.path.exists(out_file):
                    df.to_csv(os.path.join(out_path,'outliers',f'lakesStats_{outlierMethods.value}_{wqpMaps.value}.csv'),mode='a', header=False)
                else:
                    df.to_csv(os.path.join(out_path,'outliers',f'lakesStats_{outlierMethods.value}_{wqpMaps.value}.csv')) 
            except:
                # Open a file with access mode 'a'
                file_object = open(os.path.join(out_path,f'error_images_{sensor.value}.txt'), 'a')
                # Append 'hello' at the end of file
                file_object.write(src.name)
                file_object.write("\n")
                # Close the file
                file_object.close()